In [1]:
# ffmpeg -i input_video.mp4 frames/frame_%04d.png
# ffmpeg -framerate 30 -i enhanced_frames/frame_%04d.png -c:v libx264 -pix_fmt yuv420p output_video.mp4

In [2]:
import cv2
import os

In [3]:
def smooth_image(image_path, output_path, kernel_size=(5, 5) , output_dir='enhanced_frames'):
    """
    Apply Gaussian smoothing to an image to increase its quality.

    Parameters:
    image_path (str): Path to the input image.
    output_path (str): Path to save the smoothed image.
    kernel_size (tuple): Size of the Gaussian kernel. Default is (5, 5).
    """
    # Read the image
    image = cv2.imread(image_path)
    
    # Check if the image was successfully loaded
    if image is None:
        print(f"Error: Unable to load image at {image_path}")
        return
    
    # Apply Gaussian smoothing
    smoothed_image = cv2.GaussianBlur(image, kernel_size, 0)

    # Create the enhanced_frames directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    
    # Save the smoothed image
    cv2.imwrite(output_path, smoothed_image)

# print(f"smoothing is done inside output_dir folder")
# Example usage
smooth_image('frames/frame_0152.png', 'enhanced_frames/frame_0152.png')

Extract frames from the video

In [5]:
# Create a directory to store the frames
if not os.path.exists('frames'):
    os.makedirs('frames')

# Open the video file
video_path = './video/film.mp4'
cap = cv2.VideoCapture(video_path)

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = f'frames/frame_{frame_count:04d}.png'
    cv2.imwrite(frame_filename, frame)
    frame_count += 1

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

Enhance the frames using gaussian

In [ ]:
for i in range(frame_count):
    frame_filename = f'frames/frame_{i:04d}.png'
    enhanced_frame_filename = f'enhanced_frames/frame_{i:04d}.png'
    smooth_image(frame_filename, enhanced_frame_filename,kernel_size=(9,9))

Reconstruct the video from the enhanced frames

In [ ]:
# Open the original video to get its properties
original_video = cv2.VideoCapture(video_path)
fps = original_video.get(cv2.CAP_PROP_FPS)
frame_width = int(original_video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(original_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_video.release()

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use other codecs as well
out = cv2.VideoWriter('output_video.mp4', fourcc, fps, (frame_width, frame_height))
for i in range(frame_count):
    enhanced_frame_filename = f'enhanced_frames/frame_{i:04d}.png'
    enhanced_frame = cv2.imread(enhanced_frame_filename)
    
    if enhanced_frame is None:
        print(f"Error: Unable to load image at {enhanced_frame_filename}")
        continue
    
    out.write(enhanced_frame)

out.release()
cv2.destroyAllWindows()

Install conda dependencies

In [ ]:
!conda install -y -c fastai -c pytorch -c anaconda fastai pytorch torchvision

Channels:
 - fastai
 - pytorch
 - anaconda
 - defaults
 - conda-forge
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - fastai
    - pytorch
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    catalogue-2.0.10           |  py312hecd8cb5_0          40 KB  anaconda
    certifi-2024.8.30          |  py312hecd8cb5_0         166 KB  anaconda
    cloudpathlib-0.16.0        |  py312hecd8cb5_1          81 KB  anaconda
    conda-24.11.0              |  py312hecd8cb5_0         1.2 MB  anaconda
    confection-0.1.4           |  py312h8e4b320_0          82 KB  anaconda
    cymem-2.0.6                |  py312hcec6c5f_0          46 KB  anaconda
    cython-blis-0.7.11         |  py312h32608ca_0         5.6 MB  anaconda
    fastai-2.7.17              |             py_0         181 KB  fastai
    fastcore

Clone the DeOldify repository:

In [ ]:
!git clone https://github.com/jantic/DeOldify.git
!cd DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2620, done.
remote: Counting objects: 100% (274/274), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 2620 (delta 93), reused 212 (delta 73), pack-reused 2346 (from 1)
Receiving objects: 100% (2620/2620), 69.71 MiB | 1.07 MiB/s, done.
Resolving deltas: 100% (1176/1176), done.


Install additional requirements:

In [ ]:
import torch
print(torch.__version__)

2.3.1


In [ ]:
# Error will be shown for torch version 
# We already have a higher version of torch installed
!pip install -r DeOldify/requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==1.10.0 (from versions: 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==1.10.0
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Using cached wandb-0.19.0-py3-none-macosx_11_0_x86_64.whl.metadata (10 kB)
  Using cached fastai-1.0.60-py3-none-any.whl.metadata (14 kB)
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Using cached ffmpeg-1.4.tar.gz (5.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached yt_dlp-2024.12.3-py3-none-any.whl.metadata (172 kB)
  Using cached Pillow-9.3.0.tar.gz (50.4 MB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement torch==1.11.0 (from versions: 2.2.0, 2.2.1, 2.2.2)
ERROR: No matching distribution found for torch==1.11.0


Download the pre-trained models:

In [6]:
!mkdir -p DeOldify/models
#three diff models -> info in the deoldify's github
# !wget -P DeOldify/models https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth
# !wget -P DeOldify/models https://data.deepai.org/deoldify/ColorizeStable_gen.pth
!wget -P DeOldify/models https://data.deepai.org/deoldify/ColorizeVideo_gen.pth

--2024-12-05 13:49:36--  https://data.deepai.org/deoldify/ColorizeVideo_gen.pth
Resolving data.deepai.org (data.deepai.org)... 143.244.56.52
Connecting to data.deepai.org (data.deepai.org)|143.244.56.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874066230 (834M) [application/octet-stream]
Saving to: ‘DeOldify/models/ColorizeVideo_gen.pth’

ColorizeVideo_gen.p   2%[                    ]  24.62M  1.37MB/s    eta 11m 9s ^C


Set up DeOldify:

In [ ]:
import sys
sys.path.append('./DeOldify')
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)

Import necessary libraries:



In [ ]:
from deoldify.visualize import *
import fastai

Load the colorizer:

In [ ]:
# # Artistic model
# colorizer = get_image_colorizer(artistic=True)
# # Stable model
# colorizer = get_image_colorizer(artistic=False)
# Video model
video_colorizer = get_video_colorizer()

Colorize an image:

In [ ]:
# path_to_image = 'path_to_your_image.jpg'

# # Artistic model
# artistic_colorizer.plot_transformed_image(path_to_image, render_factor=35, display_render_factor=True, figsize=(8,8))

# # Stable model
# stable_colorizer.plot_transformed_image(path_to_image, render_factor=35, display_render_factor=True, figsize=(8,8))

# If we want to specify the output path
# output_path = 'path_to_save_colorized_image.jpg'
# colorizer.colorize_image(path=path_to_image, results_dir='.', render_factor=35, results_path=output_path)

Colorize a video

In [ ]:
path_to_video = 'path_to_your_video.mp4'
# # Video model
# video_colorizer.colorize_from_file_name(path_to_video)

# If we want to specify the output path
output_dir = 'path_to_save_colorized_video'
video_colorizer.colorize_from_file_name(path_to_video, render_factor=35, results_dir=output_dir)

Archives

In [ ]:
# ##download model
# from huggingface_hub import hf_hub_download

# model_path = hf_hub_download(
#     repo_id="davidramos/image-colorization", 
#     filename="model.pth" 
# )

# ##load weights

# # Clone the repository
# !git clone https://github.com/yourusername/colorizing_images.git

# # Change the working directory
# %cd colorizing_images

# ##create model instance

# from models import ColorizingNet
# model = ColorizingNet(device)
# model.load_state_dict(torch.load(model_path))


ModuleNotFoundError: No module named 'models'